In [4]:
import wfdb
import pandas as pd
import numpy as np
from pathos.multiprocessing import ProcessingPool as Pool
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [5]:
# root = '/Volumes/SanDisk SSD/physionet.org/files/mimic-iv-ecg/1.0/'

# split_csv = pd.read_csv(root + 'MIMIC-IV-ECG-Ext-Electrolytes/few_shot_splits/128shots/split1/Creatinine50912.csv')

# labels = pd.read_csv(root + 'MIMIC-IV-ECG-Ext-Electrolytes/mimiciv_ECGv1.1_hospV2.2_Creatinine50912.csv')

# train = []
# test = []
# val = []

# for index, row in split_csv.iterrows():
#     path = root + 'files/p' + f"{row['subject_id']}"[:4] + '/p' + f"{row['subject_id']}" + '/s' + f"{row['study_id']}"
    
#     signal, fields = wfdb.rdsamp(path + '/' + f"{row['study_id']}")

#     if row['split'] == 'train':
#         train.append(signal)
#     elif row['split'] == 'test':
#         test.append(signal)
#     elif row['split'] == 'val':
#         val.append(signal)

# len(train), len(val), len(test)


# faster version
def process_row(row):
    signal, fields = wfdb.rdsamp(row['path'] + '/' + str(row['study_id']))
    return row['split'], signal

root = '/Volumes/SanDisk SSD/physionet.org/files/mimic-iv-ecg/1.0/'

# Load the CSV files
split_csv = pd.read_csv(root + 'MIMIC-IV-ECG-Ext-Electrolytes/few_shot_splits/128shots/split1/Creatinine50912.csv')
labels = pd.read_csv(root + 'MIMIC-IV-ECG-Ext-Electrolytes/mimiciv_ECGv1.1_hospV2.2_Creatinine50912.csv')

# Precompute the paths
split_csv['path'] = root + 'files/p' + split_csv['subject_id'].astype(str).str[:4] + '/p' + split_csv['subject_id'].astype(str) + '/s' + split_csv['study_id'].astype(str)

# Initialize lists to store the signals
train = []
test = []
val = []

# Use pathos.multiprocessing to process rows in parallel
with Pool() as pool:
    results = pool.map(process_row, split_csv.to_dict('records'))

# Organize the results into train, test, and val lists
for split, signal in results:
    if split == 'train':
        train.append(signal)
    elif split == 'test':
        test.append(signal)
    elif split == 'val':
        val.append(signal)

len(train), len(val), len(test)

(128, 128, 10000)

In [6]:
reshaped_train = np.empty((len(train), 12), dtype=object)

for i in range(len(train)):
    for j in range(12):
        reshaped_train[i, j] = pd.Series(train[i][:][j]) # reshaping from (# of subjects, 5000, 12) to (# of subjects, 12) where each entry is a pd.series of length 5000

X_train = pd.DataFrame(reshaped_train)

null_counts = X_train.map(lambda x: x.isna().sum() if isinstance(x, pd.Series) else 0).sum().sum()

X_train = X_train.map(lambda x: x.fillna(0) if isinstance(x, pd.Series) else x) # there were NaNs in the data, which is odd, so I'm filling them with 0
print(f"Number of null values in X_train: {null_counts}")
X_train.shape

Number of null values in X_train: 0


(128, 12)

In [7]:
filtered_labels = labels[labels['study_id'].isin(split_csv[split_csv['split'] == 'train']['study_id'])]
y_train = filtered_labels['flag']
y_train = y_train.replace({'abnormal': 1, np.nan: 0}) # abnormal = 1, normal = 0
y_train.shape

/var/folders/hb/3k_51w_j5yd8v5cpqy1brfqr0000gn/T/ipykernel_1089/3176496352.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.replace({'abnormal': 1, np.nan: 0}) # abnormal = 1, normal = 0


(128,)

In [8]:
reshaped_val = np.empty((len(val), 12), dtype=object)
for i in range(len(val)):
    for j in range(12):
        reshaped_val[i, j] = pd.Series(val[i][:][j])

X_val = pd.DataFrame(reshaped_val)

null_counts = X_val.map(lambda x: x.isna().sum() if isinstance(x, pd.Series) else 0).sum().sum()

X_val = X_val.map(lambda x: x.fillna(0) if isinstance(x, pd.Series) else x) # there were NaNs in the data, which is odd, so I'm filling them with 0
print(f"Number of null values in X_val: {null_counts}")
X_val.shape

Number of null values in X_val: 0


(128, 12)

In [9]:
filtered_labels = labels[labels['study_id'].isin(split_csv[split_csv['split'] == 'val']['study_id'])]
y_val = filtered_labels['flag']
y_val = y_val.replace({'abnormal': 1, np.nan: 0})
y_val.shape

/var/folders/hb/3k_51w_j5yd8v5cpqy1brfqr0000gn/T/ipykernel_1089/577769219.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_val = y_val.replace({'abnormal': 1, np.nan: 0})


(128,)

In [10]:
reshaped_test = np.empty((len(test), 12), dtype=object)
for i in range(len(test)):
    for j in range(12):
        reshaped_test[i, j] = pd.Series(test[i][:][j])

X_test = pd.DataFrame(reshaped_test)

null_counts = X_test.map(lambda x: x.isna().sum() if isinstance(x, pd.Series) else 0).sum().sum()

X_test = X_test.map(lambda x: x.fillna(0) if isinstance(x, pd.Series) else x) # there were NaNs in the data, which is odd, so I'm filling them with 0
print(f"Number of null values in X_test: {null_counts}")
X_test.shape

Number of null values in X_test: 413


(10000, 12)

In [11]:
filtered_labels = labels[labels['study_id'].isin(split_csv[split_csv['split'] == 'test']['study_id'])]
y_test = filtered_labels['flag']
y_test = y_test.replace({'abnormal': 1, np.nan: 0})
y_test.shape

/var/folders/hb/3k_51w_j5yd8v5cpqy1brfqr0000gn/T/ipykernel_1089/3775222411.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test = y_test.replace({'abnormal': 1, np.nan: 0})


(10000,)

In [9]:
for i in range(1, 100000, 5000):
    classifier = RocketClassifier(rocket_transform='minirocket', num_kernels=10000)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    auroc = roc_auc_score(y_val, y_pred)
    print(f"Kernels: {i}, Accuracy: {accuracy:.5f}, F1 Score: {f1:.5f}, AUROC: {auroc:.5f}")
    
    if i == 1:
        scores_df = pd.DataFrame(columns=['Kernels', 'Accuracy', 'F1 Score', 'AUROC'])
    
    scores_df.loc[i] = [i, accuracy, f1, auroc]

Kernels: 1, Accuracy: 0.64844, F1 Score: 0.36620, AUROC: 0.56174
Kernels: 5001, Accuracy: 0.65625, F1 Score: 0.46341, AUROC: 0.60410
Kernels: 10001, Accuracy: 0.67188, F1 Score: 0.44737, AUROC: 0.60354
Kernels: 15001, Accuracy: 0.66406, F1 Score: 0.41096, AUROC: 0.58555
Kernels: 20001, Accuracy: 0.65625, F1 Score: 0.43590, AUROC: 0.59192
Kernels: 25001, Accuracy: 0.63281, F1 Score: 0.31884, AUROC: 0.53793
Kernels: 30001, Accuracy: 0.64062, F1 Score: 0.41026, AUROC: 0.57420
Kernels: 35001, Accuracy: 0.62500, F1 Score: 0.41463, AUROC: 0.56866
Kernels: 40001, Accuracy: 0.64062, F1 Score: 0.39474, AUROC: 0.56811
Kernels: 45001, Accuracy: 0.70312, F1 Score: 0.50000, AUROC: 0.63898
Kernels: 50001, Accuracy: 0.63281, F1 Score: 0.41975, AUROC: 0.57447
Kernels: 55001, Accuracy: 0.64844, F1 Score: 0.40000, AUROC: 0.57392
Kernels: 60001, Accuracy: 0.63281, F1 Score: 0.40506, AUROC: 0.56838
Kernels: 65001, Accuracy: 0.63281, F1 Score: 0.37333, AUROC: 0.55620
Kernels: 70001, Accuracy: 0.64062, F1 S

In [12]:
classifier = RocketClassifier(rocket_transform='minirocket', num_kernels=45001)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

auroc = roc_auc_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"AUROC: {auroc}")
print(f"F1 Score: {f1}")

Accuracy: 0.5919
AUROC: 0.511401445987569
F1 Score: 0.31145604859119286
